In [1]:
import os
import pandas as pd
import numpy as np


In [2]:
ingred = pd.read_csv('../data/ingredients.csv')
ingred.pop('Unnamed: 0')

df = np.load('../data/wordvec.npy')

# NumPy 배열을 판다스 데이터프레임으로 변환
df = np.squeeze(df, axis=0)
df = pd.DataFrame(df)

ingred

FileNotFoundError: ignored

In [ ]:
X = df

In [ ]:
y = ingred.to_numpy()

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(df, ingred, test_size=0.2, random_state=42)

X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)


train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_data = TensorDataset(X_test_tensor, y_train_tensor)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

input_dim = 300
output_dim = 50

class MultiClassModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 50)
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sig(x)
        return x

model = MultiClassModel(input_dim, output_dim)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20
for epoch in range(num_epochs):
    total_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Avg. Loss: {avg_loss:.4f}")

In [ ]:
torch.save({
    'model_state_dict': model.state_dict(),
    'model_architecture': model,
}, 'ingredient_prediction_model.pth')